Train your own word2vec representations, as you did in the first example in this checkpoint. However, you need to experiment with the hyperparameters of the vectorization step. Modify the hyperparameters and run the classification models again. Can you wrangle any improvements?


In [1]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import re
import nltk
from nltk.corpus import gutenberg
import gensim
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Utility function for standard text cleaning
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation that spaCy doesn't
    # recognize: the double dash --. Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

In [ ]:
# load and clean the data
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# the chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [ ]:
# parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [ ]:
# group into sentences
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# combine the sentences from the two novels into one data frame
sentences = pd.DataFrame(alice_sents + persuasion_sents, columns = ["text", "author"])
sentences.head()

In [ ]:
# get rid off stop words and punctuation
# and lemmatize the tokens
for i, sentence in enumerate(sentences["text"]):
    sentences.loc[i, "text"] = [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop]

In [ ]:
# train word2vec on the the sentences
model1 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=4,
    sg=0,
    sample=1e-3,
    size=100,
    hs=1
)

model2 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=6,
    sg=0,
    sample=1e-3,
    size=100,
    hs=1
)

model3 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=8,
    sg=0,
    sample=1e-3,
    size=100,
    hs=1
)

model4 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=4,
    sg=0,
    sample=1e-3,
    size=200,
    hs=1
)

model5 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=6,
    sg=0,
    sample=1e-3,
    size=200,
    hs=1
)

model6 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=8,
    sg=0,
    sample=1e-3,
    size=200,
    hs=1
)

In [ ]:
#what is going on here?

word2vec_arr1 = np.zeros((sentences.shape[0],100))
word2vec_arr2 = np.zeros((sentences.shape[0],100))
word2vec_arr3 = np.zeros((sentences.shape[0],100))
word2vec_arr4 = np.zeros((sentences.shape[0],200))
word2vec_arr5 = np.zeros((sentences.shape[0],200))
word2vec_arr6 = np.zeros((sentences.shape[0],200))

for i, sentence in enumerate(sentences["text"]):
    word2vec_arr1[i,:] = np.mean([model1[lemma] for lemma in sentence], axis=0)
    word2vec_arr2[i,:] = np.mean([model2[lemma] for lemma in sentence], axis=0)
    word2vec_arr3[i,:] = np.mean([model3[lemma] for lemma in sentence], axis=0)
    word2vec_arr4[i,:] = np.mean([model4[lemma] for lemma in sentence], axis=0)
    word2vec_arr5[i,:] = np.mean([model5[lemma] for lemma in sentence], axis=0)
    word2vec_arr6[i,:] = np.mean([model6[lemma] for lemma in sentence], axis=0)

word2vec_arr1 = pd.DataFrame(word2vec_arr1)
word2vec_arr2 = pd.DataFrame(word2vec_arr2)
word2vec_arr3 = pd.DataFrame(word2vec_arr3)
word2vec_arr4 = pd.DataFrame(word2vec_arr4)
word2vec_arr5 = pd.DataFrame(word2vec_arr5)
word2vec_arr6 = pd.DataFrame(word2vec_arr6)

sentences1 = pd.concat([sentences[["author", "text"]],word2vec_arr1], axis=1)
sentences1.dropna(inplace=True)

sentences2 = pd.concat([sentences[["author", "text"]],word2vec_arr2], axis=1)
sentences2.dropna(inplace=True)

sentences3 = pd.concat([sentences[["author", "text"]],word2vec_arr3], axis=1)
sentences3.dropna(inplace=True)

sentences4 = pd.concat([sentences[["author", "text"]],word2vec_arr4], axis=1)
sentences4.dropna(inplace=True)

sentences5 = pd.concat([sentences[["author", "text"]],word2vec_arr5], axis=1)
sentences5.dropna(inplace=True)

sentences6 = pd.concat([sentences[["author", "text"]],word2vec_arr6], axis=1)
sentences6.dropna(inplace=True)